In [1]:
import time
import requests
import pandas as pd
from tqdm import tqdm

import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver

browser = webdriver.Chrome()

browser.get('https://stockmages.netlify.app')

### To Scrapp all the data from website scroll downuntil to reach last image of the website

In [2]:
for i in tqdm(range(0,5000000,1000)):
    
    ### if we start tne new loop with range(5000000,New End,1000) and same below code than we get the more no. of images
    
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")    ### it will scroll the page automatically and working on pixel value.
    time.sleep(.1)                                                  ### it will stop the scrolling for 0.1 nsec.

100%|██████████| 5000/5000 [09:59<00:00,  8.35it/s]


### Scrapping the page source

In [2]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

### Checking number of images that has to be scraped

In [3]:
len(soup.find_all('div', class_ = 'container'))

48

### Scraping the Image Details

In [4]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):

    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link, tags, likes, comments])

100%|██████████| 48/48 [00:00<00:00, 3427.59it/s]


### Saving data in a DataFrame

In [5]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


### Checking Null Values

In [6]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

### Checking Duplicate Values

In [7]:
df['img_link'].describe()

count                                                    48
unique                                                   48
top       https://cdn.pixabay.com/photo/2022/03/06/05/30...
freq                                                      1
Name: img_link, dtype: object

### Removing Duplicate Values

In [8]:
df = df.drop_duplicates()

###  Saving the DataFrame as CSV

In [9]:
df.to_csv('images.csv', index = False)

### Downloading the images

In [10]:
import requests
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('images.csv')
df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


### Difining Dynamic Name

In [11]:
ref = 100000
for i in range(1,11):  
    print('A' + str(i + ref)[1:])

A00001
A00002
A00003
A00004
A00005
A00006
A00007
A00008
A00009
A00010


### Defining Image Downloader Function


In [12]:
def download(link, img_path ):

    res = requests.get(link)         # Sending the Request
    fd = open(img_path,'wb')         # Getting Image in WB Format
    fd.write(res.content)            # Saving the Image
    fd.close()

### Downloading all Images

In [13]:
paths   = []
img_ids = []

ref = 100000
i   = 1

for link in tqdm(df['img_link']):
    
    img_id = 'A' + str(i + ref)[1:]          # Creating Image ID
    i += 1
    
    img_path = 'Imgs1/' + img_id + '.jpg'    # Creating Image Path and creating the folder Imgs1 to store the images
    
    download(link, img_path)                # Downloading the Image
    
    paths.append(img_path)
    img_ids.append(img_id)

100%|██████████| 48/48 [00:33<00:00,  1.42it/s]


### Saving Ids and Paths in DataFrame

In [15]:
df['img_id']   = img_ids
df['img_path'] = paths

df.head()

,img_link,tags,likes,comments,img_id,img_path
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55,A00001,Imgs1/A00001.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20,A00002,Imgs1/A00002.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106,A00003,Imgs1/A00003.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11,A00004,Imgs1/A00004.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15,A00005,Imgs1/A00005.jpg


###  Saving Final Dataset

In [16]:
df.to_csv('Final_imgs.csv', index = False)